In [2]:
from netCDF4 import Dataset
import numpy as np 

In [33]:
testpath = '/home/tom/Documents/UC/Project/data/dap.ceda.ac.uk/neodc/esacci/fire/data/burned_area/MODIS/grid/v5.1/2004'
testfile = '20040301-ESACCI-L4_FIRE-BA-MODIS-fv5.1.nc'

rootgrp = Dataset(testpath + '/' + testfile, "r", format="NETCDF4")

print(rootgrp)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: Fire_cci Gridded MODIS Burned Area product
    institution: University of Alcala
    source: MODIS MOD09GQ Collection 6, MODIS MOD09GA Collection 6, MODIS MCD14ML Collection 6, ESA CCI Land Cover dataset v2.0.7
    history: Created on 2018-12-01 03:23:27
    references: See www.esa-fire-cci.org
    tracking_id: e06b5b77-6829-4731-a034-bcc8f991c5f6
    Conventions: CF-1.6
    product_version: v5.1
    summary: The grid product is the result of summing up burned area pixels and their attributes, as extracted from their original sinusoidal projection, within each cell of 0.25 degrees in a regular grid covering the whole Earth in monthly composites. The attributes stored are sum of burned area, standard error, fraction of burnable area, fraction of observed area, number of patches and the burned area for 18 land cover classes of Land Cover CCI.
    keywords: Burned Area, Fire Disturb

In [9]:
def lat_to_idx(latitude):
    return (90 - latitude)*4

def lon_to_idx(longitude):
    return (-180 - longitude)*-4

In [35]:
var_name = 'lat_bnds'
var = rootgrp.variables[var_name]

sth_kor_lat_min = 32
sth_kor_lat_max = 39
sth_kor_lon_min = 125
sth_kor_lon_max = 130

#print(var[lat_to_idx(sth_kor_lat_max):lat_to_idx(sth_kor_lat_min)])

var_name = 'lon_bnds'
var = rootgrp.variables[var_name]

#print(np.asarray(var))
#print(var[lon_to_idx(sth_kor_lon_min):lon_to_idx(sth_kor_lon_max)])

var_name = 'burned_area'
var = rootgrp.variables[var_name]

print(np.array(var[:, lat_to_idx(sth_kor_lat_max):lat_to_idx(sth_kor_lat_min), lon_to_idx(sth_kor_lon_min):lon_to_idx(sth_kor_lon_max)]))
print(np.array(var[:, lat_to_idx(sth_kor_lat_max):lat_to_idx(sth_kor_lat_min), lon_to_idx(sth_kor_lon_min):lon_to_idx(sth_kor_lon_max)]).shape)

[[[       0.        0.        0.        0.        0.        0.        0.
          0.        0.        0. 26832334. 30105878.        0.        0.
          0.        0.        0.        0.        0.        0.]
  [       0.        0.        0.        0.        0.        0.        0.
          0.        0.  3971185.        0.        0.  2468574.  1019628.
          0.        0.        0.        0.        0.        0.]
  [       0.        0.        0.        0.        0.        0.        0.
     375652. 36116320.   536646.  7298394.   268323.  6869077.        0.
          0.        0.        0.        0.        0.        0.]
  [       0.        0.        0.        0.        0.        0.        0.
    5098143. 12020885.        0.        0.        0.        0.        0.
          0.        0.        0.        0.        0.        0.]
  [       0.        0.        0.        0.        0.        0.        0.
          0.        0.        0.        0.        0.        0.        0.
          0.  